## Convert KITTI Point Cloud to BEV image

In [ ]:
from pathlib import Path
from easydict import EasyDict
from tqdm import tqdm
import pickle
import yaml
import cv2
from itertools import product

from pcdet.utils.bev_utils import get_sample, pointcloud3d_to_bevimage2d, show_bev_image_preview, generate_groundtruth_dicts_bev
from pcdet.utils.bev_utils import boxes3d_lidar_to_rotated_bev_boxes, draw_bev_boxes, save_bev_images_and_boxes


dataset_cfg = EasyDict(yaml.safe_load(open('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/dataset_configs/kitti_dataset_bev_custom.yaml')))
class_names = ['Car', 'Pedestrian', 'Cyclist']
file_path = '/home/rlab10/OpenPCDet/lidar2bev/master_thesis/lidar3d_to_bev2d.ipynb' 
NOTEBOOK_DIR = (Path(file_path).resolve().parent).resolve()
ROOT_DIR = Path('/home/rlab10/OpenPCDet')
DATA_VARIANT = dataset_cfg.DATA_VARIANT
MODE = 'train' if dataset_cfg.DATA_SPLIT['train'] else 'val'

data_path_kitti = ROOT_DIR / 'data' / 'kitti' # raw data path
save_path_kitti = NOTEBOOK_DIR / 'kitti' / MODE

box_colormap = [
    [1, 1, 1], # not assigned
    [0, 1, 0], # Car (green)
    [1, 0, 1], # Pedestrian (violet)
    [0, 1, 1], # Cyclist (yellow)
]

pkl_paths = {
    'train': {
        'default': data_path_kitti / dataset_cfg.PKL_TRAIN_DEFAULT,
        'densified': data_path_kitti / dataset_cfg.PKL_TRAIN_DENSIFIED,
        'upsampled': data_path_kitti / dataset_cfg.PKL_TRAIN_UPSAMPLED,
        'interpolated': data_path_kitti / dataset_cfg.PKL_TRAIN_INTERPOLATED,
    },
    'val': {
        'default': data_path_kitti / dataset_cfg.PKL_VAL_DEFAULT,
        'densified': data_path_kitti / dataset_cfg.PKL_VAL_DENSIFIED,
        'upsampled': data_path_kitti / dataset_cfg.PKL_VAL_UPSAMPLED,
        'interpolated': data_path_kitti / dataset_cfg.PKL_VAL_INTERPOLATED,
    }
}

pkl_path = pkl_paths[MODE][DATA_VARIANT]
compression_params = [cv2.IMWRITE_PNG_COMPRESSION, 0] # no compression



def main(visualize_samples=False):
    print(f"Mode: {MODE}, Data Variant: {DATA_VARIANT}, Saving: {dataset_cfg.SAVE_FLAG}")
    print(f"Load: {pkl_path}")
    with open(pkl_path, 'rb') as f:
        data_list = pickle.load(f)

    num_samples = len(data_list)
    # Original sample + 6 Augmentations else no augmentations
    num_augments = 7 if MODE == 'train' else 1 
    print(f"Samples: {num_samples}")
    print(f"Augmentations: {num_augments}")
    print(f"All: {num_samples * num_augments}")

    # DEBUG - single samples
    data_list = [data_list[4]]

    # main loop
    for sample_idx in tqdm(range(len(data_list)), desc="Samples"):
        for augment_idx in range(num_augments):
            sample = get_sample(data=data_list, sample_idx=sample_idx, augment_idx=augment_idx, mode=MODE, class_names=class_names)
            print(sample['frame_id'])
            bev_image_bgr, _ = pointcloud3d_to_bevimage2d(points=sample['points'], cfg=dataset_cfg, num_slices=dataset_cfg.NUM_SLICES, 
                                                      filter_points=False)
            bev_image_bgr_vis = bev_image_bgr.copy()
            
            valid_bev_boxes, valid_indices = [], []
            if 'gt_boxes' in sample:
                for i in range(len(sample['gt_boxes'])):
                    boxes3d = sample['gt_boxes'][i]
                    bev_box = boxes3d_lidar_to_rotated_bev_boxes(
                        bev_img=bev_image_bgr,
                        bev_image_height=dataset_cfg.BEV_IMAGE_HEIGHT,
                        bev_image_width=dataset_cfg.BEV_IMAGE_WIDTH,
                        bev_res=dataset_cfg.CELL_SIZE,
                        boxes3d=boxes3d,
                        min_points=5
                    )                     
                    if bev_box is not None:
                        draw_bev_boxes(bev_image=bev_image_bgr_vis, bev_box=bev_box[:-1], cls_idx=int(bev_box[-1]), box_colormap=box_colormap, 
                                    thickness=1, obj_direction_color=(0,255,255))
                        valid_bev_boxes.append(bev_box)
                        valid_indices.append(i)

            if visualize_samples:
                key = show_bev_image_preview(bev_image=bev_image_bgr_vis, BGR2RGB=True, win_size=(800, 800))
            else:
                key = None

            if valid_bev_boxes and dataset_cfg.SAVE_FLAG:
                save_bev_images_and_boxes(save_path=save_path_kitti, frame_id=sample['frame_id'], bev_image_bgr=bev_image_bgr, 
                                        valid_bev_boxes=valid_bev_boxes, augment_idx=augment_idx, compression=compression_params,
                                        normalize_coords=dataset_cfg.NORM_BOX_COORDS, data_variant=DATA_VARIANT, mode=MODE)
                
                if MODE == 'val' and dataset_cfg.SAVE_FLAG and DATA_VARIANT == 'default':
                    generate_groundtruth_dicts_bev(sample, valid_indices, output_path=save_path_kitti / 'val_gt_data', frame_id=sample['frame_id'])

            if key == ord('q'):
                break
        if key == ord('q'):
            break
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main(visualize_samples=False)

## Diff Check between Annotations and BEV Images

In [ ]:
import os
import re

# Ordner und die zugehörige Dateiendung
bev_dirs = [
    # ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_images_default', '.png'),
    # ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_images_densified', '.png'),
    # ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_images_upsampled', '.png'),
    # ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_images_interpolated', '.png'),
    # ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_annos', '.txt'),
    # ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/val_gt_data', '.txt'),
    # check seperately
    ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/train/bev_images_default', '.png'),
    ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/train/bev_images_densified', '.png'),
    ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/train/bev_images_upsampled', '.png'),
    ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/train/bev_images_interpolated', '.png'),
    ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/train/bev_annos', '.txt'),
]

#bev_dirs = [
    #('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/zod/val/bev_images_default', '.png'),
    #('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/zod/val/bev_annos', '.txt'),
    # check seperately
    #('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/zod/train/bev_images_default', '.png'),
    #('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/zod/train/bev_annos', '.txt'),
#]

def extract_frame_id(filename):
    # Extrahiere frame_id vor einem eventuellen Augment-Tag und Dateiendung
    match = re.match(r"(\d+)", filename)
    return match.group(1) if match else None

frame_id_sets = []
for bev_dir, ext in bev_dirs:
    frame_ids = set()
    for fname in os.listdir(bev_dir):
        if fname.endswith(ext):
            frame_id = extract_frame_id(fname)
            if frame_id:
                frame_ids.add(frame_id)
    frame_id_sets.append(frame_ids)
    print(f"{bev_dir}: {len(frame_ids)} unique frame_ids")

# Finde frame_ids, die nicht in allen Verzeichnissen vorkommen
all_frame_ids = set.union(*frame_id_sets)
common_frame_ids = set.intersection(*frame_id_sets)
not_in_all = all_frame_ids - common_frame_ids

print("\nFrame IDs, die NICHT in allen Verzeichnissen vorkommen:")
for fid in sorted(not_in_all):
    missing = [os.path.basename(d[0]) for d, s in zip(bev_dirs, frame_id_sets) if fid not in s]
    print(f"{fid} fehlt in: {', '.join(missing)}")

/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_images_default: 3675 unique frame_ids
/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_images_densified: 3675 unique frame_ids
/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_images_upsampled: 3675 unique frame_ids
/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_images_interpolated: 3675 unique frame_ids
/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_annos: 3675 unique frame_ids
/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/val_gt_data: 3675 unique frame_ids

Frame IDs, die NICHT in allen Verzeichnissen vorkommen:


## Convert ZOD Point Cloud to BEV image

In [ ]:
from pathlib import Path
from easydict import EasyDict
from tqdm import tqdm
import pickle
import yaml
import cv2

from pcdet.utils.bev_utils import get_sample, pointcloud3d_to_bevimage2d, show_bev_image_preview,  generate_groundtruth_dicts_bev
from pcdet.utils.bev_utils import boxes3d_lidar_to_rotated_bev_boxes, draw_bev_boxes, save_bev_images_and_boxes


dataset_cfg = EasyDict(yaml.safe_load(open('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/dataset_configs/zod_dataset_bev_custom.yaml')))
class_names = ['Car', 'Pedestrian', 'Cyclist']
file_path = '/home/rlab10/OpenPCDet/lidar2bev/master_thesis/lidar3d_to_bev2d.ipynb' 
NOTEBOOK_DIR = (Path(file_path).resolve().parent).resolve()
ROOT_DIR = Path('/home/rlab10/OpenPCDet')
DATA_VARIANT = dataset_cfg.DATA_VARIANT
MODE = 'train' if dataset_cfg.DATA_SPLIT['train'] else 'val'

data_path_zod = ROOT_DIR / 'data' / 'zod' # raw data path
save_path_zod = NOTEBOOK_DIR / 'zod' / MODE

box_colormap = [
    [1, 1, 1], # not assigned
    [0, 1, 0], # Car (green)
    [1, 0, 1], # Pedestrian (violet)
    [0, 1, 1], # Cyclist (yellow)
]

pkl_paths = {
    'train': {
        'default': data_path_zod / dataset_cfg.PKL_TRAIN_DEFAULT,
    },
    'val': {
        'default': data_path_zod / dataset_cfg.PKL_VAL_DEFAULT,
    }
}

pkl_path = pkl_paths[MODE][DATA_VARIANT]
compression_params = [cv2.IMWRITE_PNG_COMPRESSION, 0] # no compression



def main(visualize_samples=False):
    print(f"Mode: {MODE}, Data Variant: {DATA_VARIANT}, Saving: {dataset_cfg.SAVE_FLAG}")
    print(f"Load: {pkl_path}")
    with open(pkl_path, 'rb') as f:
        data_list = pickle.load(f)

    num_samples = len(data_list)
    # Original sample + 6 Augmentations else no augmentations
    num_augments = 7 if MODE == 'train' else 1 
    print(f"Samples: {num_samples}")
    print(f"Augmentations: {num_augments}")
    print(f"All: {num_samples * num_augments}")

    # DEBUG - single samples
    data_list = [data_list[0]]

    # main loop
    for sample_idx in tqdm(range(len(data_list)), desc="Samples"):
        for augment_idx in tqdm(range(num_augments), disable=True):
            sample = get_sample(data=data_list, sample_idx=sample_idx, augment_idx=augment_idx, mode=MODE, class_names=class_names)
            print(sample['frame_id'])
            bev_image_bgr, _ = pointcloud3d_to_bevimage2d(points=sample['points'], cfg=dataset_cfg, num_slices=dataset_cfg.NUM_SLICES, 
                                                      filter_points=False)
            bev_image_bgr_vis = bev_image_bgr.copy()
            
            valid_bev_boxes, valid_indices = [], []
            if 'gt_boxes' in sample:
                for i in range(len(sample['gt_boxes'])):
                    boxes3d = sample['gt_boxes'][i]
                    bev_box = boxes3d_lidar_to_rotated_bev_boxes(
                        bev_img=bev_image_bgr,
                        bev_image_height=dataset_cfg.BEV_IMAGE_HEIGHT,
                        bev_image_width=dataset_cfg.BEV_IMAGE_WIDTH,
                        bev_res=dataset_cfg.CELL_SIZE,
                        boxes3d=boxes3d,
                        min_points=5
                    )
                    if bev_box is not None:
                        draw_bev_boxes(bev_image=bev_image_bgr_vis, bev_box=bev_box[:-1], cls_idx=int(bev_box[-1]), box_colormap=box_colormap, 
                                       thickness=1, obj_direction_color=(0,255,255))
                        valid_bev_boxes.append(bev_box)
                        valid_indices.append(i)

            if visualize_samples:
                key = show_bev_image_preview(bev_image=bev_image_bgr_vis, BGR2RGB=True, win_size=(800, 800))
            else:
                key = None

            if valid_bev_boxes and dataset_cfg.SAVE_FLAG:
                save_bev_images_and_boxes(save_path=save_path_zod, frame_id=sample['frame_id'], bev_image_bgr=bev_image_bgr, 
                                          valid_bev_boxes=valid_bev_boxes, augment_idx=augment_idx, compression=compression_params,
                                          normalize_coords=dataset_cfg.NORM_BOX_COORDS, data_variant=DATA_VARIANT, mode=MODE)
                
                if MODE == 'val' and dataset_cfg.SAVE_FLAG:
                    generate_groundtruth_dicts_bev(sample, valid_indices, output_path=save_path_zod / 'val_gt_data', frame_id=sample['frame_id'])

            if key == ord('q'):
                break
        if key == ord('q'):
            break
    cv2.destroyAllWindows()
    

if __name__ == '__main__':
    main(visualize_samples=True)